In [ ]:
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib widget
%matplotlib inline
    
import pandas as pd
from mpl_toolkits import mplot3d
#from mpl_toolkits.mplot3d import Axes3D

import numpy as np
# Table 3: males, Table 4: females
deaths_m = pd.read_excel ("deathsyoauk2017tablesfinal.xls", nrows=106, usecols="B:BD", sheet_name="Table 3", skiprows=[0,1,2])
deaths_f = pd.read_excel ("deathsyoauk2017tablesfinal.xls", nrows=106, usecols="B:BD", sheet_name="Table 4", skiprows=[0,1,2])

#deaths = pd.read_excel ("https://bit.ly/2OVxChn", nrows=106, usecols="B:BD", sheet_name="Table 3", skiprows=[0,1,2])
#deaths = pd.read_excel ("https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathregistrationssummarytablesenglandandwalesdeathsbysingleyearofagetables/2017/deathsyoauk2017tablesfinal.xls", nrows=106, usecols="B:BD", sheet_name="Table 3", skiprows=[0,1,2])

# has suffix in table 1: deaths.rename (columns={"2017p": 2017}, inplace=True) # also need this in read_excel: usecols="B:AS", 

#print (deaths.columns, deaths.index, deaths.values)

In [ ]:


#print (deaths.shape)
def z_function(x, y):
    return np.sin(np.sqrt(x ** 2 + y ** 2))

def z_f (deaths, x, y):
    ages, time = x.shape
    #print (f"{ages=}, {time=}")
    ret = np.full (shape=(ages, time), fill_value=0.0)  #len(x), len(y)), fill_value=0.0) #shape = (len(x), len (y)), fill_value = 0.0)
    #print ("zf", len (x), len (y), ret.shape)
    #print (len(x), x, len(y), y)
    #print (f"{ret.shape=}")
    for i in range (ages): #len (x)):
        for j in range (time): #0, 54):
#            print ("setting {}, {} to {}".format (i, j, deaths.iloc[j,i]))
            try:
                #print (deaths.iloc [i, j])
                ret [i, j] = deaths.iloc [i, j] #pass #print (x)
            except Exception as e:
                print ("broken", i, j, ret.shape, deaths.shape, e)
        
    return ret

def mangle_df (deaths):
    x= deaths.columns #[1:] # columns 0 is the header for Ages
    y= deaths.index # index[0] is the header
    #print (f"{x=}, {y=}, {deaths.columns=}, {deaths.index=}")
    X, Y = np.meshgrid(x, y)
    Z = z_f (deaths, X, Y)
    return X, Y, Z

X, Y, Z = mangle_df (deaths_m)
X1,Y1,Z1= mangle_df (deaths_f)

In [ ]:
d = pd.read_excel ("annualdeaths2024.xlsx", sheet_name="Table_3", skiprows=[0,1,2,3])
d.columns = ['Year', 'Sex', 'Age', 'Marital status', 'Number of deaths']

all_ppl = d[(d['Marital status'] == "All marital statuses") & (d['Age'].apply (lambda x: 'All' not in str(x)))]

all_ppl.loc [:, 'Age'] = all_ppl.loc[:, 'Age'].apply (lambda x: int(str(x).split()[0]))
#all_ppl ['Age'] = [int (i.split()[0]) for i in all_ppl['Age'] if (not isinstance (i, int)) & (not i.startswith ('All'))] # tidy up '105 and over'

males = all_ppl [all_ppl['Sex'] == 'Male']
females = all_ppl [all_ppl['Sex'] == 'Female']
#dd = pd.ExcelFile ("annualdeaths2024.xlsx")
#print (dd.sheet_names)
#print ([i for i in dir (d) if "sheet" in i.lower()])
#later_deaths = pd.read_excel ("annualdeaths2024.xlsx", sheet_name="Table 3", skiprows=[0,1,2])
#later_deaths

# check consistency with older data
for yr in range (2015, 2018):
    for age in range (106):
        if not np.array_equal (males[(males['Year']==yr) & (males['Age']==age)]['Number of deaths'].values, deaths [deaths.index == age][yr].values):
            print (yr, age)
#print (all_ppl)

In [ ]:
# nasty kludge to tack on the data from 2018 to 2024

males.iloc[:,[0,2,4]]
ys=males.iloc[:, 0].values
len(ys)
len(ys[ys>2017]) 
ys[:7*106]


len(X), len(Y), len(Z)
X.shape, Y.shape, Z.shape, X1.shape, Y1.shape, Z1.shape, X2.shape, Y2.shape, Z2.shape

a, b = np.meshgrid (females.iloc[:, 0].values [:106], females.iloc[:, 2].values[:7])

years = np.array (range (2018, 2025))
ages = np.array (range (106))
#a, b = np.meshgrid (*[np.array (sorted (set (females.iloc[:, j].values))) for j in [0,2]])
a, b = np.meshgrid (years, ages)
a, b, X, Y, females[(females['Year']==2017) & (females['Age']==3)].iloc[:,4]
def zeds_from_ppl (ppl):
    return np.array ([[ppl[(ppl ['Year'] == a[r, c]) & (ppl['Age'] == b[r,c])].iloc[:,4].values[0] for r in range (a.shape[0])] for c in range(a.shape[1])]).T

zeds_from_ppl (females)
# not really necessary as X? are the same as are Y?, a, b
X2 = np.append (X, a, axis=1)
Y2 = np.append (Y, b, axis=1)
Z2 = np.append (Z, zeds_from_ppl (males), axis=1)

X3 = np.append (X1, a, axis=1)
Y3 = np.append (Y1, b, axis=1)
Z3 = np.append (Z1, zeds_from_ppl (females), axis=1)

#Z.shape, zeds_from_ppl(males).shape, Z[0,:], zeds_from_ppl(males)[0,:]

# Evolution of Age at Death for England Tells many Stories

The plot below is intended to be interactive: you can zoom in, out and around it to get a feel for the overall shape and anomalies.

This quite simple presentation of how the age at death has evolved can support a number of interesting stories, including:

## Those not born during WWI and WWII
The grooves of those not born due to soldiers at war, and the corresponding peaks from home comings carve through the ageing peaks

## How dangerous the first year of life used to be
The first year of life is still much more dangerous than the following decades, but it used to be much worse and was until the early '70s the most dangerous age to be.

## The slight uptick of risk for adolescents and early adulthood

## Although middle ages has got considerably safer, life has not extended
The trailing edge of the curve gets increasingly steep. Unfortunately for many, much of the extension is much more about living with poor health conditions, rather than enjoying later life.

## Averages can be misleading
Taking the early annual snapshots, especially, illustrates how misleading 'life expectancy' for a population can be. Although it's not easy to trace the actual age at death for any give cohort - there aren't enough years covered to do this - a good approximation can be deduced by slicing along a year. This exercise illustrates how it is possible for the expected life span for a given group to be skewed by infant death in particular, and early deaths in general.

In [ ]:
# from plotly example

# Import dependencies
import plotly
#import plotly.graph_objs as go_s  looks like these are the same. Unsure which is newer
import plotly.graph_objects as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()


# I tried Mesh3d for this: interpolates a solid around the values. scatter_3d doesn't join the dots
#figure = go.Figure (data=[go.Surface (x=X, y=Y, z=Z, opacity=0.25, colorscale=[[0, 'red'], [1,'red']]),
#                          go.Surface (x=X1,y=Y1,z=Z1,opacity=0.25, colorscale=[[0, 'blue'],[1,'blue']])])
figure = go.Figure (data=[go.Surface (x=X2, y=Y2, z=Z2, opacity=0.25, colorscale=[[0, 'red'], [1,'red']]),
                          go.Surface (x=X3, y=Y3, z=Z3,opacity=0.25, colorscale=[[0, 'blue'],[1,'blue']])])

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

#data = [trace]

#plot_figure = go.Figure(data=data, layout=layout)

figure.update_layout(
    title=dict(
        text="Evolution of Age at Death. Males: red, Females: blue"
    ),
    width=1200, height=1000,
    showlegend=True,
    scene = dict (
        xaxis = dict (title= "year"),
        yaxis = dict (title="age"),
        zaxis = dict (title="count")
    ),
    #zaxis = dict (title="z title"),
    legend=dict(
        title=dict(
            text="Legend Title"
        )
    ),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

#plotly.offline.iplot(plot_figure)
plotly.offline.iplot(figure)

In [ ]:
deaths_m = pd.read_excel ("deathsyoauk2017tablesfinal.xls", nrows=106, usecols="B:BD", sheet_name="Table 3", skiprows=[0,1,2])
print (deaths_m.values[0,:])


In [ ]:
figure = go.Figure (data=[go.Surface (z=net, opacity=0.5)])
figure.update_layout(
    title=dict(
        text="Ratio of Deaths by Age over Time"
    ),
    width=1200, height=1000,
    scene = dict (
        xaxis = dict (title= "year"),
        yaxis = dict (title="age"),
        zaxis = dict (title="count")
    ),
    #zaxis = dict (title="z title"),
    legend=dict(
        title=dict(
            text="Legend Title"
        )
    ),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
plotly.offline.iplot(figure)

In [ ]:
(X[1], X[2], Y[1], len(Z[1]), len(X), len(Y), len(Z), len(X[0]), len(Y[0]))
X.shape, Y.shape, Z.shape


In [ ]:
males[(males['Year']==2015) & (males['Age']==105)]

In [ ]:
deaths[deaths.index==105][2015]

In [ ]:
'105'.split()

In [ ]:
len(males)

In [ ]:
males[males['Year'].isin(range (2018, 2025))]